In [1]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from random import seed
from random import randrange, uniform

import import_ipynb

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [2]:
objectives_path = 'CSVs/companies/companies.csv'
result_path = 'CSVs/linkedin_about/companies_about.csv'

clutch_companies_path = 'CSVs/clutch/clutch_companies.csv'
linkedin_accounts_path = 'CSVs/commons/accounts.csv'

In [3]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--incognito")

In [4]:
seed(1)

In [5]:
def get_companies_df():
    try: return pd.read_csv(objectives_path)
    except: return pd.DataFrame()

In [6]:
def get_companies_result_df():
    try: return pd.read_csv(result_path)
    except: return pd.DataFrame()

In [7]:
def get_linkedin_accounts():
    return None

In [8]:
def get_linkedin_active_account():
    username = 'zeipf10@gmail.com'
    password = 'dextra#2021'
    return username, password

In [9]:
def auth_linkedin(browser):
    
    linkedin_login_url = 'https://www.linkedin.com/uas/login'
    
    try:
        print('Accessing: ', linkedin_login_url)
        browser.get(linkedin_login_url)
        time.sleep(uniform(1,2))
    
        username, password = get_linkedin_active_account()
    
        print('Sign up with: ', username)
        username_field = browser.find_element_by_id('username')
        username_field.send_keys(username)

        password_field = browser.find_element_by_id('password')
        password_field.send_keys(password)
    
        time.sleep(uniform(1,2))
        password_field.submit()
    
    except:
        print('ERROR: AUTH LINKEDIN FAILS')

In [10]:
def get_company_id(company):
    
    linkedin_search_url = 'https://www.linkedin.com/search/results/companies/?keywords=' + company
    print('Accessing: ', linkedin_search_url)
    browser.get(linkedin_search_url)
    time.sleep(uniform(1,2))
    
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    code = soup.find_all('code')

    try:
        for code_str in code:
            
            if('urn:li:fsd_company:' in code_str.text):
                
                company_id = re.findall(r"(?<=fsd_company:)\d+?(?=[,|\"])", code_str.text)[0]
                
                if company_id == '' or company_id == '8175': 
                    return ''
                
        return company_id
    
    except:
        print('ERROR: - Company ID except for: ', company)
        return ''

In [11]:
def append_empty(company):
    info_to_append = {  'company_searched': company, 
                        'company_found': '', 
                        'found': False, 
                        'company_id': '', 
                        'employees': '',
                        'industry': '',
                        'specialties': '',
                        'headquarters': ''  }
    
    df = get_companies_result_df()
    df = df.append(info_to_append, ignore_index=True)
    df.to_csv(result_path, index=False)

In [12]:
def find_company_title_on(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
        company_title = company_title.find("span").getText()
        return company_title
    except:
        try:
            time.sleep(uniform(1,2))
            company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
            company_title = company_title.find("span").getText()
            return company_title
        except:
            print('ERROR: - Company Title except')
            return 'NF'

In [13]:
def find_company_employees_on(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"})
        employees = employees.getText().replace('\n', '').strip()
        return employees
    except:
        try:
            time.sleep(uniform(1,2))
            employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"})
            employees = employees.getText().replace('\n', '').strip()
            return employees
        except:
            print('ERROR: - Company Employees except')
            return 'NF'

In [14]:
def find_company_details_dict_on(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        company_details_dict = {}
        company_details = soup.find('dl', attrs={'class': 'overflow-hidden'}).find_all()
        for num, info in enumerate(company_details):
            if(info.name == 'dt'):
                company_details_dict[info.text.replace('\n', '').strip()] = company_details[num+1].text.replace('\n', '').strip()
        return company_details_dict
    except:
        print('ERROR: - Company Details except')
        return {}

In [15]:
def find_total_jobs_on(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        total_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text)
        total_jobs = total_jobs.group(1).strip()
        return total_jobs
    except:
        print('ERROR: - Company Total Jobs except')
        return '0'

In [16]:
def find_usa_iteng_jobs_on(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        iteng_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text)
        iteng_jobs = iteng_jobs.group(1).strip()
        return iteng_jobs
    except:
        print('ERROR: - Company IT-ENG Jobs except')
        iteng_jobs = '0'

In [17]:
def get_employees_number(companies):
    
    for processed, company in enumerate(companies):
        
        print('Iteração :', processed, ' de ', len(companies))
        company_id = get_company_id(company)

        if company_id == '':
            print('**Company id empty')
            append_empty(company)
            continue
        else:
            print('**Founded some company id')
            pass
        
        linkedin_url = "https://www.linkedin.com/company/" + company_id + "/about/"
        
        #accessing the company's page
        print('Acessing: ', linkedin_url)
        browser.get(linkedin_url)
        time.sleep(uniform(3,6))
        
        #searching for the company title
        company_title = find_company_title_on(browser)
        
        #searching for the number of employees
        employees = find_company_employees_on(browser)
        
        #searching for the company info
        company_details_dict = find_company_details_dict_on(browser)
        
        #searching for total jobs count
        linkedin_total_url = 'https://www.linkedin.com/jobs/search/?geoId=92000000&f_C=' + company_id
        print('Acessing: ', linkedin_total_url)
        browser.get(linkedin_total_url)
        time.sleep(uniform(3,6))
        
        total_jobs = find_total_jobs_on(browser)
        
        if total_jobs != '0':
            #searching for it and engeneering jobs count
            linkedin_iteng_url = 'https://www.linkedin.com/jobs/search/?f_C=' + company_id + '&f_CR=103644278&f_F=it%2Ceng&geoId=92000000&sortBy=R'
            print('Acessing: ', linkedin_iteng_url)
            browser.get(linkedin_iteng_url)
            time.sleep(uniform(3,6))
        
            iteng_jobs = find_usa_iteng_jobs_on(browser)
        else:
            iteng_jobs = '0'
        
        info_to_append = {  'company_searched': company,
                            'company_found': company_title, 
                            'found': True, 
                            'company_id': company_id,
                            'linkedin': linkedin_url,
                            'total_jobs':total_jobs,
                            'it_eng_jobs': iteng_jobs,
                            'website': company_details_dict.get('Site', 'NF'),
                            'employees': company_details_dict.get('Tamanho da empresa', 'NF'),
                            'industry': company_details_dict.get('Setor', 'NF'),
                            'specialties': company_details_dict.get('Especializações', 'NF'),
                            'headquarters': company_details_dict.get('Sede', 'NF')  }
        
        
        df = get_companies_result_df()
        df = df.append(info_to_append, ignore_index=True)
        df.to_csv(result_path, index=False)
        
        #storing company data found
        print('-----------DADOS-----------')
        print('company_searched', company)
        print('company_found', company_title) 
        print('company_id', company_id)
        print('linkedin', linkedin_url)
        print('total_jobs', total_jobs)
        print('iteng_jobs', iteng_jobs)
        print('website', company_details_dict.get('Site', ''))
        print('employees', company_details_dict.get('Tamanho da empresa', ''))
        print('industry', company_details_dict.get('Setor', ''))
        print('specialties', company_details_dict.get('Especializações', ''))
        print('headquarters', company_details_dict.get('Sede', ''))
        print('---------------------------------------------')

In [18]:
def get_company_list():
    try:
        companies = get_companies_df()['company'].to_list()
        return list(filter(lambda elm: isinstance(elm, str), companies))
    except:
        print('get_company_list except')
        return []

In [19]:
def get_company_searched():
    try:
        searched = pd.read_csv(result_path)['company_searched'].to_list()
        return list(filter(lambda elm: isinstance(elm, str), searched))
    except:
        print('get_company_searched except')
        return []

In [24]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


In [25]:
auth_linkedin(browser)

Accessing:  https://www.linkedin.com/uas/login
Sign up with:  zeipf10@gmail.com


In [26]:
companies = get_company_list()
companies_searched = get_company_searched()
to_search = [c for c in companies if c not in companies_searched]
    
print('De: ', len(companies))
print('Restam: ', len(to_search))

De:  3830
Restam:  2114


In [ ]:
to_search_count = 1

while to_search_count > 0:
    
    companies = get_company_list()
    companies_searched = get_company_searched()
    to_search = [c for c in companies if c not in companies_searched]
    
    to_search_count = len(to_search)    
    count = min(randrange(40,70), to_search_count)
    
    print('--- De: ', len(companies))
    print('--- Restam: ', len(to_search))
    print("Buscando mais:", count)
    
    get_employees_number(to_search[0:count])

    timeSleep = randrange(30,60)
    
    print("Esperando um total de:", timeSleep)
    time.sleep(timeSleep)

--- De:  3830
--- Restam:  2114
Buscando mais: 58
Iteração : 0  de  58
Accessing:  https://www.linkedin.com/search/results/companies/?keywords=Human Resources
**Founded some company id
Acessing:  https://www.linkedin.com/company/1185120/about/
Acessing:  https://www.linkedin.com/jobs/search/?geoId=92000000&f_C=1185120
Acessing:  https://www.linkedin.com/jobs/search/?f_C=1185120&f_CR=103644278&f_F=it%2Ceng&geoId=92000000&sortBy=R
-----------DADOS-----------
company_searched Human Resources
company_found Human Resources
company_id 1185120
linkedin https://www.linkedin.com/company/1185120/about/
total_jobs 15
iteng_jobs 3
website 
employees 11-50 funcionários
industry Recursos humanos
specialties 
headquarters Ciudad Autónoma de Buenos Aires, Buenos Aires
---------------------------------------------
Iteração : 1  de  58
Accessing:  https://www.linkedin.com/search/results/companies/?keywords=Humana Inc.
**Founded some company id
Acessing:  https://www.linkedin.com/company/4257/about/
Aces

In [ ]:
get_company_searched()

In [ ]:
get_company_searched()['found'].value_counts()